In [1]:
import numpy as np 
import os
import copy
from math import *
import matplotlib.pyplot as plt
from functools import reduce
# reading in dicom files
import pydicom
# skimage image processing packages
from skimage import measure, morphology
from skimage.morphology import ball, binary_closing
from skimage.measure import label, regionprops
# scipy linear algebra functions 
from scipy.linalg import norm
import scipy.ndimage
# plotly 3D interactive graphs 
import plotly
from plotly.graph_objs import *
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [5]:
def get_dicom_files(folder):
    files_in_folder = os.listdir(folder)
    files_in_folder = [folder + i for i in files_in_folder]
    return files_in_folder

def load_scan(path): 
    '''Load all DICOM images from a folder into a list for manipulation.
    Infer slice thickness '''
    slices = [pydicom.read_file(path+'/'+ s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(scans):
    '''Pixels outside cylindrical scanning bounds are set to Hounsfield 
    Units of air. 
    Convert to Hounsfield units by multiplying a rescale slope and adding
    an intercept from teh metadata of scans.
    '''
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)
    counts = counts[vals != bg]
    vals = vals[vals != bg]
    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None
def segment_lung_mask(image, fill_lung_structures=True):
    ''' Isolate just the lungs and some tissue around it by masking
    out everything else'''
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image >= -700, dtype=np.int8)+1
    labels = measure.label(binary_image)
 
    # Pick the pixel in the very corner to determine which label is     air.
    # Improvement: Pick multiple background labels from around the    patient
    # More resistant to “trays” on which the patient lays cutting   the air around the person in half
    background_label = labels[0,0,0]
 
    # Fill the air around the person
    binary_image[background_label == labels] = 2
 
    # Method of filling the lung structures (that is superior to 
    # something like morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
 
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
 
    # Remove other air pockets inside body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image


In [5]:
def get_dicom_files(folder):
    files_in_folder = os.listdir(folder)
    files_in_folder = [folder + i for i in files_in_folder]
    return files_in_folder

train = 'C:\\Users\\lilyx\\Kaggle_OSIC\\data\\train\\'
file_folders = get_dicom_files(train)
# file_folders[:3]

# set path and load files 
path = file_folders[3]
# patient_dicom = load_scan(path)
# patient_pixels = get_pixels_hu(patient_dicom)

# # get masks 
# segmented_lungs = segment_lung_mask(patient_pixels, fill_lung_structures=False)
# segmented_lungs_fill = segment_lung_mask(patient_pixels, fill_lung_structures=True)
# internal_structures = segmented_lungs_fill - segmented_lungs
# # isolate lung from chest
# copied_pixels = copy.deepcopy(patient_pixels)
# for i, mask in enumerate(segmented_lungs_fill): 
#     get_high_vals = mask == 0
#     copied_pixels[i][get_high_vals] = 0
# seg_lung_pixels = copied_pixels
# plt.imshow(seg_lung_pixels[326], cmap=plt.cm.bone)

for patient in file_folders[:1]:
    try:
        slices = [pydicom.read_file(path+'/'+ s) for s in os.listdir(path)]
        plt.imshow(slices[0].pixel_array)
        plt.show()
    except:
        continue
#     ds = pydicom.dcmread(file_folders[777]) # File Reading/Parsing 
#     plt.imshow(ds.pixel_array, cmap=plt.cm.bone) 

In [12]:
# file_folders
import tensorflow as tf
import numpy as np
IMG_SIZE_PX = 50
SLICE_COUNT = 20
n_classes = 2

In [15]:
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='VALID')

def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='VALID')



def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,5,1,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([13*13*5*64,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 13*13*5*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

def train_neural_network(x):
    
    much_data = np.load('C:\\Users\\lilyx\\Kaggle_OSIC\\data\\train_for_CNN_50.npy', allow_pickle=True)
    train_data = much_data[:-200]
    validataion_data = much_data[-200:]
    
    prediction = convolutional_neural_network(x)
    cost = tf.nn.l2_loss(prediction,y)
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 3
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for data in train_data:
                X = data[0]
                Y = data[1]
                _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))

train_neural_network(x)

OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.